In [ ]:
!pip install scikit-learn pandas joblib

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib

df = pd.read_csv("adult.csv")
df.columns = df.columns.str.strip()
df = df.replace(' ?', pd.NA).dropna()

X = df[[
    'age', 'education', 'educational-num', 'marital-status',
    'occupation', 'gender', 'hours-per-week', 'capital-gain', 'capital-loss'
]]
y = df['income']

categorical_features = ['education', 'occupation', 'gender', 'marital-status']
numerical_features = ['age', 'educational-num', 'hours-per-week', 'capital-gain', 'capital-loss']

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('num', StandardScaler(), numerical_features)
])

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "GradientBoosting": GradientBoostingClassifier()
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results = {}
best_model = None
best_score = 0
best_name = ""

for name, model in models.items():
    pipe = Pipeline([
        ('preprocessing', preprocessor),
        ('model', model)
    ])

    print(f"🔍 Training {name}...")
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    results[name] = acc

    print(f"✅ {name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))
    print("=" * 60)

    if acc > best_score:
        best_score = acc
        best_model = pipe
        best_name = name

if best_model:
    joblib.dump(best_model, "best_model.pkl")
    print(f"🏆 Best Model: {best_name} with accuracy {best_score:.4f}")
    print("📦 Saved as: best_model.pkl")


🔍 Training LogisticRegression...
✅ LogisticRegression Accuracy: 0.8540
              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.91      7479
        >50K       0.73      0.59      0.66      2290

    accuracy                           0.85      9769
   macro avg       0.81      0.76      0.78      9769
weighted avg       0.85      0.85      0.85      9769

🔍 Training RandomForest...
✅ RandomForest Accuracy: 0.8487
              precision    recall  f1-score   support

       <=50K       0.89      0.91      0.90      7479
        >50K       0.69      0.64      0.66      2290

    accuracy                           0.85      9769
   macro avg       0.79      0.78      0.78      9769
weighted avg       0.84      0.85      0.85      9769

🔍 Training KNN...
✅ KNN Accuracy: 0.8414
              precision    recall  f1-score   support

       <=50K       0.88      0.91      0.90      7479
        >50K       0.68      0.61      0.64      2290

    accurac

In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

model = joblib.load("best_model.pkl")

st.set_page_config(page_title="Salary Prediction", page_icon="💼", layout="centered")
st.title("💼 Employee Salary Prediction")
st.markdown("Predict whether an employee earns >50K or ≤50K based on their details.")

st.sidebar.header("Input Employee Details")

age = st.sidebar.slider("Age", 18, 65, 30)
education = st.sidebar.selectbox("Education", [
    "Bachelors", "HS-grad", "11th", "Masters", "9th", "Some-college", "Assoc-acdm", "Assoc-voc",
    "7th-8th", "Doctorate", "Prof-school", "5th-6th", "10th", "1st-4th", "Preschool", "12th"
])
marital_status = st.sidebar.selectbox("Marital Status", [
    "Married-civ-spouse", "Divorced", "Never-married", "Separated", "Widowed", "Married-spouse-absent", "Married-AF-spouse"
])
occupation = st.sidebar.selectbox("Occupation", [
    "Tech-support", "Craft-repair", "Other-service", "Sales", "Exec-managerial", "Prof-specialty",
    "Handlers-cleaners", "Machine-op-inspct", "Adm-clerical", "Farming-fishing",
    "Transport-moving", "Priv-house-serv", "Protective-serv", "Armed-Forces"
])
gender = st.sidebar.selectbox("Gender", ["Male", "Female"])
hours_per_week = st.sidebar.slider("Hours per Week", 1, 99, 40)
educational_num = st.sidebar.slider("Education Num", 1, 16, 10)
capital_gain = st.sidebar.number_input("Capital Gain", 0, 99999, 0)
capital_loss = st.sidebar.number_input("Capital Loss", 0, 99999, 0)

input_df = pd.DataFrame({
    'age': [age],
    'education': [education],
    'educational-num': [educational_num],
    'marital-status': [marital_status],
    'occupation': [occupation],
    'gender': [gender],
    'hours-per-week': [hours_per_week],
    'capital-gain': [capital_gain],
    'capital-loss': [capital_loss]
})

st.write("### 👤 Input Preview")
st.dataframe(input_df)

if st.button("🔮 Predict Income Class"):
    prediction = model.predict(input_df)[0]
    st.success(f"🧠 Prediction: The person is likely to earn **{prediction}**")

st.markdown("---")
st.markdown("#### 📂 Batch Prediction with Optional Ground Truth")

uploaded_file = st.file_uploader("Upload a CSV file for batch prediction", type="csv")

if uploaded_file is not None:
    batch_data = pd.read_csv(uploaded_file)

    actual_labels = None
    if 'income' in batch_data.columns:
        actual_labels = batch_data['income']
        batch_data = batch_data.drop(columns=['income'])
    expected_cols = ['age', 'education', 'educational-num', 'marital-status',
                 'occupation', 'gender', 'hours-per-week', 'capital-gain', 'capital-loss']

    for col in expected_cols:
        if col not in batch_data.columns:
            batch_data[col] = 0
    batch_data = batch_data[expected_cols]

    preds = model.predict(batch_data)
    batch_data['PredictedClass'] = preds

    if actual_labels is not None:
        batch_data['ActualClass'] = actual_labels
        batch_data['Correct'] = batch_data['PredictedClass'] == batch_data['ActualClass']
        accuracy = batch_data['Correct'].mean()

        st.success(f"✅ Accuracy on uploaded data: **{accuracy:.2%}**")

        correct_counts = batch_data['Correct'].value_counts()
        st.write("### 🧠 Prediction Correctness")
        st.pyplot(correct_counts.plot.pie(autopct="%1.1f%%", labels=["Correct", "Incorrect"], figsize=(4, 4), title="Prediction Accuracy").figure)

        st.write("### 📊 Predicted vs Actual Class Distribution")
        if 'ActualClass' in batch_data.columns:
            class_dist = pd.DataFrame({
                "Predicted": batch_data['PredictedClass'].value_counts(),
                "Actual": actual_labels.value_counts()
            })
            st.bar_chart(class_dist)

    else:
        st.warning("⚠️ No `income` column found for ground truth comparison.")

    st.write("✅ Predictions:")
    st.dataframe(batch_data)

    csv = batch_data.to_csv(index=False).encode('utf-8')
    st.download_button("📥 Download Predictions CSV", csv, file_name='predicted_results.csv', mime='text/csv')


Overwriting app.py


In [2]:
!pip install streamlit pyngrok --quiet

from pyngrok import ngrok

!ngrok config add-authtoken 3031mIuH47ewY72cRzOnw4lYlXV_3ByUxRVcEPzigRMTF9CqL

!streamlit run app.py &> /dev/null &

public_url = ngrok.connect(8501)
print("🔗 Streamlit app is live at:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔗 Streamlit app is live at: NgrokTunnel: "https://5d189d869d97.ngrok-free.app" -> "http://localhost:8501"
